## Importing the Data

In [1]:
# Constants
DATASET_DIR = './data/'
GLOVE_DIR = './glove.6B/'
SAVE_DIR = './'

import os
import pandas as pd

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [25]:
X.head(1000)

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8
5,6,1,"Dear @LOCATION1, I think that computers have a...",8
6,7,1,Did you know that more and more people these d...,10
7,8,1,@PERCENT1 of people agree that computers make ...,10
8,9,1,"Dear reader, @ORGANIZATION1 has had a dramatic...",9
9,10,1,In the @LOCATION1 we have the technology of a ...,9


In [32]:
X.iloc[3255]['essay']

"I do not belive that they should be takein off the shelfs beacuse @CAPS1 people still come to get that sort of stuff. @CAPS1 people say is not plite for little kids 'well keek your kids away from it then.'             @CAPS1 people dont like it and @CAPS1 people do. I am one of the people that like that kind of stuff, beacuse no one can tell us what we can and cant watch or say. It is our life if we wanna watch that kind of stuff or read about it we can.            Noone is telling people that they half to read it or even look at it. If you dont like it dont look at it. @CAPS2 kind of stuff should be off to the side where you know what your goin to look at and away from kids                       @CAPS2 shouldnt even be a problem. @CAPS2 is just people that think they are high class and think if you look at that or watch that or read that you are a dirt bag. That is there opinon.             In conclusion  I think if you dont read @CAPS2 stuff or watch @CAPS2 stuff you are trying to b

Minimum and Maximum Scores for each essay set.

In [3]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

## Preprocessing the Data

We will preprocess all essays and convert them to feature vectors so that they can be fed into the RNN.

These are all helper functions used to clean the essays.

In [4]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [5]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

Using TensorFlow backend.


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rutvikdixit32/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rutvikdixit32/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Training Phase

Now we train the model on the dataset.

We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold.
We will then calculate Average Kappa for all the folds.

In [8]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [9]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

cv = KFold(n_splits=5, shuffle=True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 8 models.
    if count == 5:
         lstm_model.save('./model_weights/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1
        


--------Fold 1--------

Training Word2Vec Model...


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10380/10380 [==============================] - 9s 832us/step - loss: 63.7886 - mean_absolute_error: 4.3162
Epoch 2/50
10380/10380 [==============================] - 5s 478us/step - loss: 39.7148 - mean_absolute_error: 3.5063
Epoch 3/50
10380/10380 [========

10381/10381 [==============================] - 5s 493us/step - loss: 14.5601 - mean_absolute_error: 2.1887
Epoch 16/50
10381/10381 [==============================] - 5s 485us/step - loss: 13.7533 - mean_absolute_error: 2.1261
Epoch 17/50
10381/10381 [==============================] - 5s 494us/step - loss: 13.5126 - mean_absolute_error: 2.1248
Epoch 18/50
10381/10381 [==============================] - 5s 489us/step - loss: 12.8947 - mean_absolute_error: 2.0630
Epoch 19/50
10381/10381 [==============================] - 5s 494us/step - loss: 12.3072 - mean_absolute_error: 2.0116
Epoch 20/50
10381/10381 [==============================] - 5s 496us/step - loss: 11.9767 - mean_absolute_error: 1.9673
Epoch 21/50
10381/10381 [==============================] - 5s 495us/step - loss: 11.9718 - mean_absolute_error: 1.9472
Epoch 22/50
10381/10381 [==============================] - 5s 491us/step - loss: 11.6310 - mean_absolute_error: 1.9348
Epoch 23/50
10381/10381 [==============================] - 5

10381/10381 [==============================] - 5s 505us/step - loss: 8.4577 - mean_absolute_error: 1.5915
Epoch 38/50
10381/10381 [==============================] - 5s 505us/step - loss: 8.4044 - mean_absolute_error: 1.5859
Epoch 39/50
10381/10381 [==============================] - 5s 499us/step - loss: 7.8931 - mean_absolute_error: 1.5758
Epoch 40/50
10381/10381 [==============================] - 5s 502us/step - loss: 7.8238 - mean_absolute_error: 1.5550
Epoch 41/50
10381/10381 [==============================] - 5s 506us/step - loss: 7.9200 - mean_absolute_error: 1.5647
Epoch 42/50
10381/10381 [==============================] - 5s 505us/step - loss: 7.4706 - mean_absolute_error: 1.5325
Epoch 43/50
10381/10381 [==============================] - 5s 502us/step - loss: 7.9065 - mean_absolute_error: 1.5473
Epoch 44/50
10381/10381 [==============================] - 5s 501us/step - loss: 7.7885 - mean_absolute_error: 1.5346
Epoch 45/50
10381/10381 [==============================] - 5s 504us/

The Avg. Kappa Score is 0.961 which is the highest we have ever seen on this dataset.

In [11]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.9608
